In [1]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np

# load modules related to this exercise
from model_zucher_exante import zurcher
from Solve_NFXP_exante import solve_NFXP
import estimate_NFXP_exante as estimate

# Exercise 1

#### 1. Look at ReadMe.txt to get an overview of the code

#### 2. Invistigate how the code works, that is ensure you understand:
<il type ="a">
<li> zurcher.init</li>
<li> zurcher.setup</li>
<li> zurcher.create_grid</li>
<li> zucher.state_transition </li>
<li> zucher.bellman </li>

You can see how they are called below
    

#### 3. Fill in the missing stuff in the function zucher.bellman and run the code below

In [ ]:
do_settings = {
    'RC': 0.5,
    'n': 12,
    'p':[0.65,0.2,0.1]   
}
model = zurcher(**do_settings)

print('Model grid:\n',model.grid)
print('Transition probabilities conditional on not replacing:\n',model.P1)
print('Transition probabilities conditional on replacing:\n',model.P2)
ev,pk, dev = model.bellman(np.zeros((model.n)),output=3)
print('Bellman one run:\n',ev)



    
### 4. Solve the model. In order to solve the model, you should understand:
<li> solve_NFXP.init</li>
<li> solve_NFXP.setup</li>
<li> solve_NFXP.poly </li>
<li> solve_NFXP.sa </li>
<li> solve_NFXP.nk </li>
</il>
You can see how they are called below: 

In [ ]:
algorithm = 'poly'
do_settings_solver = {
    'sa_min': 10,
    'sa_max': 20,  
    'printfxp': 2
}

solver = solve_NFXP(**do_settings_solver)
model = zurcher()

if algorithm == 'sa':
    ev = solver.sa(model.bellman)
if algorithm == 'poly':
    ev = solver.poly(model.bellman)
else:
    print('Algorithm must be "sa" or "poly"')



#### 5. Now we have to estimate the model. In order to estimate the model, you should understand:
<il type ="a">
<li> zurcher.read_busdata </li>
<li> estimate_NFXP.estimate  </li>
<li> estimate_NFXP.ll  </li>
</il>

You can see how they are called below:

#### 6. Fill in the missing stuff in the function estimate_NFXP.ll, and estimate the model to check that your results are correct

In [ ]:
# Set up the model
model = zurcher()

# Set-up solver
solver = solve_NFXP()

# Read the data
data = model.read_busdata(bustypes=[1,2,3,4])
samplesize = data.shape[0]

# Estimate the model
import time
t0 = time.time()
theta0 = [0,0]

# args for nfxp estimate
nfxp_model, optim_res, pnames, theta_hat, Avar, converged=estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

t1 = time.time()
time = t1-t0

# Print the result
print(f'Structual estimation using busdata from Rust(1987)')
print(f'Beta        = {model.beta:.4f}')
print(f'n           = {model.n}')
print(f'Sample size = {samplesize}\n \n')

print(f'Parameters     Estimates    s.e. ') 
print(f'{pnames[0]}             {theta_hat[0]:.4f}     {np.sqrt(Avar[0,0]):.4f} ')
print(f'{pnames[1]}              {theta_hat[1]:.4f}     {np.sqrt(Avar[1,1]):.4f} \n ')
print(f'{pnames[2]}(1)           {theta_hat[2]:.4f}     {np.sqrt(Avar[2,2]):.4f}  ')
print(f'{pnames[2]}(2)           {theta_hat[3]:.4f}     {np.sqrt(Avar[3,3]):.4f}  ')
print(f'{pnames[2]}(3)           {theta_hat[4]:.4f}     {np.sqrt(Avar[4,4]):.4f}  ')
print(f'{pnames[2]}(4)           {theta_hat[5]:.4f}     {np.sqrt(Avar[5,5]):.4f}  \n')


print(f'Log-likelihood {-optim_res.fun*samplesize:.2f}') 
print(f'runtime (seconds) {time:.4f}')
print(f'The model converged: {converged}')

#### 7. Try using line_profiler in python. This gives you a lot of information about the performance of your code

In [ ]:
%lprun -f estimate.ll  -f estimate.estimate estimate.estimate(model, solver,data,theta0=theta0, twostep=0)

In [ ]:
%lprun -f solve_NFXP.nk -f solve_NFXP.poly solve_NFXP.poly(solver,model.bellman)

a) Now try changing the optimizer options, and turn the use of the non-numerical Hessian off . What happens?

b) Now also try it with the analytical gradient off, what happens?

#### 8. Try estimate the model for different values of $\beta$. 

(a) Why can we not estimate $\beta$?

(b) When estimating with different $\beta$, do the changes in the estimates of c and/or RC make intuitively sense?

(c) Can you think of some data/variation, which could allow us to identify $\beta$?

#### 9. We use the latest EV guess to start the solve-procedure even though we change $\theta$ from one likelihood iteration to another. Why do you think we do that? 
(a) What if we started over with EV=0 each iteration? Try that and see what happens with the parameters and the numerical performance.

#### 10. Try setting the maximum number of miles (odometer reading) to 900. Now the absorbing state is much higher. 

(a) If we adjust the number of grid points as well, so that we have a comparable model (multiply the number of grids by 2), do we get a better fit? 

(b) Try to lower the number of grid points to 175 again. How do the parameters change? Are the changes intuitive? 

(c) What if you change the max to 225 and half the number of grids (hint: what goes wrong?)?